In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/sample_submission.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/subjects.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tasks.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/daily_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/48b636e0f5.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/9fb7805d99.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/e658b0aa3d.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/1c3719ea59.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/164adaed7b.parquet
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/unlabeled/24

In [2]:
!pip install tsflex --no-index --find-links=file:///kaggle/input/kagglelib
!pip install seglearn --no-index --find-links=file:///kaggle/input/kagglelib

Looking in links: file:///kaggle/input/kagglelib
Processing /kaggle/input/kagglelib/tsflex-0.3.0-py3-none-any.whl
Looking in links: file:///kaggle/input/kagglelib
Processing /kaggle/input/kagglelib/seglearn-1.2.5-py3-none-any.whl


In [3]:
import glob
from tqdm.auto import tqdm
from sklearn import *
from seglearn.feature_functions import base_features, emg_features
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors
from tsflex.features.integrations import seglearn_feature_dict_wrapper
import pathlib
import xgboost as xgb
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import uniform, randint
from sklearn.metrics import average_precision_score, make_scorer

In [4]:
# Set the base path
base_path = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

# Get the file paths for train and test data
train_files = glob.glob(os.path.join(base_path, 'train/**/**'))
test_files = glob.glob(os.path.join(base_path, 'test/**/**'))

# Load the metadata files
tdcsfog_metadata = pd.read_csv(os.path.join(base_path, 'tdcsfog_metadata.csv'))
defog_metadata = pd.read_csv(os.path.join(base_path, 'defog_metadata.csv'))
metadata = pd.concat([tdcsfog_metadata, defog_metadata])
# Load the subjects file
subjects = pd.read_csv(os.path.join(base_path, 'subjects.csv'))

# Load the tasks file
tasks = pd.read_csv(os.path.join(base_path, 'tasks.csv'))

# Load the sample submission file
sub = pd.read_csv(os.path.join(base_path, 'sample_submission.csv'))

In [5]:
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[-1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['t_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(tasks[tasks.columns[1:]])

subjects = subjects.fillna(0).groupby('Subject').median()
subjects = subjects.reset_index()
# subjects.rename(columns={'Subject':'Id'}, inplace=True)
subjects['s_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(subjects[subjects.columns[1:]])
subjects=subjects.rename(columns={'Visit':'s_Visit','Age':'s_Age','YearsSinceDx':'s_YearsSinceDx','UPDRSIII_On':'s_UPDRSIII_On','UPDRSIII_Off':'s_UPDRSIII_Off','NFOGQ':'s_NFOGQ'})

In [6]:
tasks

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST,t_kmeans
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320,2
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698,0
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299,3
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351,7
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960,3
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889,3
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152,3
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831,7


In [7]:
complex_featlist=['Visit','Test','Medication','s_Visit','s_Age','s_YearsSinceDx','s_UPDRSIII_On','s_UPDRSIII_Off','s_NFOGQ','s_kmeans']
metadata_complex=metadata.merge(subjects,how='left',on='Subject').copy()
metadata_complex['Medication']=metadata_complex['Medication'].factorize()[0]

In [8]:
metadata_complex

,Id,Subject,Visit,Test,Medication,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,s_kmeans
0,003f117e14,13abfd,3,2.0,0,0.0,68.0,9.0,17.0,15.0,15.0,8
1,009ee11563,d81e3a,4,2.0,0,0.0,71.0,10.0,42.0,0.0,24.0,1
2,011322847a,203e85,2,2.0,0,0.0,67.0,12.0,27.0,28.0,19.0,0
3,01d0fe7266,203e85,2,1.0,1,0.0,67.0,12.0,27.0,28.0,19.0,0
4,024418ba39,cecfb8,19,3.0,0,0.0,60.0,7.0,30.0,0.0,19.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,ce8b0b,1,NaN,1,1.5,65.0,7.0,50.0,59.5,24.5,2
966,f40e8c6ebe,d9529b,1,NaN,1,1.0,28.0,4.0,54.0,50.0,25.0,9
967,f8ddbdd98d,fc1e1b,1,NaN,0,1.0,82.0,11.0,38.0,42.0,21.0,2
968,f9efef91fb,fe5d84,2,NaN,1,1.5,72.0,14.0,22.5,39.0,16.0,0


In [9]:
basic_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(base_features()),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[5_000],
    strides=[5_000],
)

emg_feats = emg_features()
del emg_feats['simple square integral'] # is same as abs_energy (which is in base_features)

emg_feats = MultipleFeatureDescriptors(
    functions=seglearn_feature_dict_wrapper(emg_feats),
    series_names=['AccV', 'AccML', 'AccAP'],
    windows=[5_000],
    strides=[5_000],
)

fc = FeatureCollection([basic_feats, emg_feats])

In [10]:
#import pathlib
def reader(f):
    try:
        df = pd.read_csv(f, index_col="Time", usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])
        
        df['Id'] = f.split('/')[-1].split('.')[0]
        df['Module'] = pathlib.Path(f).parts[-2]
        
        df['Time_frac']=(df.index/df.index.max()).values#currently the index of data is actually "Time"
        
        df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
#         df = pd.merge(df, subjects[['Id','s_kmeans']], how='left', on='Id').fillna(-1)
        df = pd.merge(df, metadata_complex[['Id','Subject']+['Visit','Test','Medication','s_kmeans']], how='left', on='Id').fillna(-1)
        df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        df = df.merge(df_feats, how="left", left_index=True, right_index=True)
        df.fillna(method="ffill", inplace=True)
        return df
    except: pass


In [11]:
train_files = pd.concat([reader(file) for file in tqdm(train_files)]).fillna(0); 
print(train_files.shape)
cols = [c for c in train_files.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
pcols = ['StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']

  0%|          | 0/970 [00:00<?, ?it/s]

(20588374, 72)


In [12]:
len(scols)

4

In [13]:
train_files=train_files.reset_index(drop=True)

In [14]:
best_params_ = {'estimator__colsample_bytree': 0.5, 'estimator__learning_rate': 0.2, 'estimator__max_depth': 10, 
 'estimator__min_child_weight': 3, 'estimator__n_estimators': 30,  'estimator__subsample': 1}
best_params_ = {pp: v for p, v in best_params_.items() for pp in p.split('__')}; del best_params_['estimator']

In [15]:
from sklearn.base import clone

def custom_average_precision(y_true, y_pred):
    score = average_precision_score(y_true, y_pred)
    return 'average_precision', score, True

class LGBMMultiOutputRegressor(MultiOutputRegressor):
    def fit(self, X, y, eval_set=None, **fit_params):
        self.estimators_ = [clone(self.estimator) for _ in range(y.shape[1])]
        
        for i, estimator in enumerate(self.estimators_):
            if eval_set:
                fit_params['eval_set'] = [(eval_set[0], eval_set[1][:, i])]
            estimator.fit(X, y[:, i], **fit_params)
        
        return self

In [16]:
from sklearn.model_selection import GroupKFold

N_FOLDS=5
kfold = GroupKFold(N_FOLDS)
group_var = train_files.Subject
groups=kfold.split(train_files, groups=group_var)
regs=[]
cvs=[]
for fold, (tr_idx,te_idx ) in enumerate(tqdm(groups, total=N_FOLDS, desc="Folds")):
    tr_idx=pd.Series(tr_idx).sample(n=2000000,random_state=42).values #2000000
    
    # Create a base XGBoost regressor with the common parameters
    base_regressor = lgb.LGBMRegressor(**best_params_)

    # Wrap the base regressor with the MultiOutputRegressor
    multioutput_regressor = LGBMMultiOutputRegressor(base_regressor)

    x_tr,y_tr=train_files.loc[tr_idx,cols].to_numpy(),train_files.loc[tr_idx,pcols].to_numpy()
    x_te,y_te=train_files.loc[te_idx,cols].to_numpy(),train_files.loc[te_idx,pcols].to_numpy()

    multioutput_regressor.fit(
    x_tr,y_tr,
    eval_set=(x_te,y_te),
    eval_metric=custom_average_precision,
    early_stopping_rounds=25
    )
    regs.append(multioutput_regressor)
    cv=metrics.average_precision_score(y_te, multioutput_regressor.predict(x_te).clip(0.0,1.0))
    cvs.append(cv)
print(cvs)

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.0578757	valid_0's average_precision: 0.130233
[2]	valid_0's l2: 0.0568393	valid_0's average_precision: 0.360881
[3]	valid_0's l2: 0.0568937	valid_0's average_precision: 0.32333
[4]	valid_0's l2: 0.0569729	valid_0's average_precision: 0.236865
[5]	valid_0's l2: 0.0565647	valid_0's average_precision: 0.292237
[6]	valid_0's l2: 0.0563773	valid_0's average_precision: 0.291197
[7]	valid_0's l2: 0.0564142	valid_0's average_precision: 0.288112
[8]	valid_0's l2: 0.0564494	valid_0's average_precision: 0.270473
[9]	valid_0's l2: 0.0564891	valid_0's average_precision: 0.248869
[10]	valid_0's l2: 0.0563303	valid_0's average_precision: 0.199375
[11]	valid_0's l2: 0.0563565	valid_0's average_precision: 0.188289
[12]	valid_0's l2: 0.0563726	valid_0's average_precision: 0.182079
[13]	valid_0's l2: 0.0563778	valid_0's average_precision: 0.181759
[14]	valid_0's l2: 0.0563302	valid_0's average_precision: 0.180522
[15]	valid_0's l2: 0.055521	valid_0's average_precision: 0.210562
[16]	v

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.192534	valid_0's average_precision: 0.401311
[2]	valid_0's l2: 0.183146	valid_0's average_precision: 0.456269
[3]	valid_0's l2: 0.174623	valid_0's average_precision: 0.495045
[4]	valid_0's l2: 0.168875	valid_0's average_precision: 0.54542
[5]	valid_0's l2: 0.168542	valid_0's average_precision: 0.543299
[6]	valid_0's l2: 0.163028	valid_0's average_precision: 0.566166
[7]	valid_0's l2: 0.155626	valid_0's average_precision: 0.603903
[8]	valid_0's l2: 0.147036	valid_0's average_precision: 0.638213
[9]	valid_0's l2: 0.146667	valid_0's average_precision: 0.635359
[10]	valid_0's l2: 0.146701	valid_0's average_precision: 0.633939
[11]	valid_0's l2: 0.144611	valid_0's average_precision: 0.638406
[12]	valid_0's l2: 0.140066	valid_0's average_precision: 0.64921
[13]	valid_0's l2: 0.139948	valid_0's average_precision: 0.644734
[14]	valid_0's l2: 0.139463	valid_0's average_precision: 0.648201
[15]	valid_0's l2: 0.135746	valid_0's average_precision: 0.665645
[16]	valid_0's l2: 0.

...

[1]	valid_0's l2: 0.000653185	valid_0's average_precision: 0.00044468
[2]	valid_0's l2: 0.000609592	valid_0's average_precision: 0.0034996
[3]	valid_0's l2: 0.000725825	valid_0's average_precision: 0.00201993
[4]	valid_0's l2: 0.000822284	valid_0's average_precision: 0.00193567
[5]	valid_0's l2: 0.000854178	valid_0's average_precision: 0.00351268
[6]	valid_0's l2: 0.000872308	valid_0's average_precision: 0.00349387
[7]	valid_0's l2: 0.000948873	valid_0's average_precision: 0.00374412
[8]	valid_0's l2: 0.00100741	valid_0's average_precision: 0.00403111
[9]	valid_0's l2: 0.00101176	valid_0's average_precision: 0.00369498
[10]	valid_0's l2: 0.00101095	valid_0's average_precision: 0.00361589
[11]	valid_0's l2: 0.00107835	valid_0's average_precision: 0.00357703
[12]	valid_0's l2: 0.00107904	valid_0's average_precision: 0.00356118
[13]	valid_0's l2: 0.00108224	valid_0's average_precision: 0.00366746
[14]	valid_0's l2: 0.00108805	valid_0's average_precision: 0.00344487
[15]	valid_0's l2: 0.00

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.0482308	valid_0's average_precision: 0.122564
[2]	valid_0's l2: 0.0465905	valid_0's average_precision: 0.130785
[3]	valid_0's l2: 0.0462238	valid_0's average_precision: 0.123464
[4]	valid_0's l2: 0.045816	valid_0's average_precision: 0.128908
[5]	valid_0's l2: 0.0454631	valid_0's average_precision: 0.137689
[6]	valid_0's l2: 0.0453136	valid_0's average_precision: 0.143603
[7]	valid_0's l2: 0.0440213	valid_0's average_precision: 0.165769
[8]	valid_0's l2: 0.04377	valid_0's average_precision: 0.172747
[9]	valid_0's l2: 0.0435837	valid_0's average_precision: 0.179535
[10]	valid_0's l2: 0.0436359	valid_0's average_precision: 0.181958
[11]	valid_0's l2: 0.0437507	valid_0's average_precision: 0.182792
[12]	valid_0's l2: 0.0438049	valid_0's average_precision: 0.186891
[13]	valid_0's l2: 0.043834	valid_0's average_precision: 0.188775
[14]	valid_0's l2: 0.043799	valid_0's average_precision: 0.190745
[15]	valid_0's l2: 0.0437701	valid_0's average_precision: 0.196805
[16]	vali

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.00437416	valid_0's average_precision: 0.00385485
[2]	valid_0's l2: 0.00432861	valid_0's average_precision: 0.00424554
[3]	valid_0's l2: 0.00436863	valid_0's average_precision: 0.0060113
[4]	valid_0's l2: 0.00439231	valid_0's average_precision: 0.00590593
[5]	valid_0's l2: 0.00438036	valid_0's average_precision: 0.005802
[6]	valid_0's l2: 0.00439906	valid_0's average_precision: 0.00561918
[7]	valid_0's l2: 0.0044095	valid_0's average_precision: 0.00563707
[8]	valid_0's l2: 0.00445638	valid_0's average_precision: 0.00596444
[9]	valid_0's l2: 0.00447454	valid_0's average_precision: 0.00594162
[10]	valid_0's l2: 0.00447175	valid_0's average_precision: 0.00797505
[11]	valid_0's l2: 0.00447025	valid_0's average_precision: 0.00798366
[12]	valid_0's l2: 0.00451825	valid_0's average_precision: 0.00836501
[13]	valid_0's l2: 0.00456169	valid_0's average_precision: 0.00812454
[14]	valid_0's l2: 0.0045948	valid_0's average_precision: 0.00770749
[15]	valid_0's l2: 0.00460106	vali

In [17]:
sub['t'] = 0
submission = []
for f in test_files:
    df = pd.read_csv(f)
    df.set_index('Time', drop=True, inplace=True)
    df['Id'] = f.split('/')[-1].split('.')[0]
    df['Time_frac']=(df.index/df.index.max()).values#currently the index of data is actually "Time"
    df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
    df = pd.merge(df, metadata_complex[['Id','Subject']+['Visit','Test','Medication','s_kmeans']], how='left', on='Id').fillna(-1)
    df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin")
    df = df.merge(df_feats, how="left", left_index=True, right_index=True)
    df.fillna(method="ffill", inplace=True)
    
    res_vals=[]
    for i_fold in range(N_FOLDS):
        res_val=np.round(regs[i_fold].predict(df[cols]).clip(0.0,1.0),3)
        res_vals.append(np.expand_dims(res_val,axis=2))
    res_vals=np.mean(np.concatenate(res_vals,axis=2),axis=2)
    res = pd.DataFrame(res_vals, columns=pcols)
    
    df = pd.concat([df,res], axis=1)
    df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
    submission.append(df[scols])


In [18]:
submission = pd.concat(submission)
submission = pd.merge(sub[['Id']], submission, how='left', on='Id').fillna(0.0)
submission[scols].to_csv('submission.csv', index=False)

In [19]:
submission

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.0124,0.0154,0.0130
1,003f117e14_1,0.0124,0.0154,0.0130
2,003f117e14_2,0.0124,0.0154,0.0130
3,003f117e14_3,0.0124,0.0154,0.0130
4,003f117e14_4,0.0124,0.0154,0.0130
...,...,...,...,...
286365,02ab235146_281683,0.0112,0.0334,0.0126
286366,02ab235146_281684,0.0112,0.0334,0.0126
286367,02ab235146_281685,0.0112,0.0334,0.0126
286368,02ab235146_281686,0.0112,0.0334,0.0126
